In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import json

In [2]:
#function to count the number of parameters
def get_n_params(model): 
    np = 0
    for p in list(model.parameters()):
        np+= p.nelement()
    return np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/malavika/anaconda3/envs/petitRT/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
pwd

'/home/malavika/Documents/Research/assessment/training'

In [5]:
path_to_file = "/home/malavika/Documents/Research/assessment/datasets/6params_train_bce_1M2.h5"
#path_to_file = "/home/malavika/Documents/Research/assessment/datasets/10params_train_bce_1M2.h5"

params = '6'

with h5py.File(path_to_file, 'r') as hdf:
    data = np.array(hdf.get('data')).astype(np.float32)
    label = np.array(hdf.get('labels')).astype(np.float32)

hdf.close()

In [6]:
np.shape(data)

(221852, 377)

In [7]:
# Normalize
a_min = np.min(data)
a_max = np.max(data)
data_norm = (data-a_min)/(a_max-a_min)

#Tensor
target = torch.from_numpy(label)
inpt = torch.from_numpy(data_norm)

class MyDataset(Dataset):
    def __init__(self):
        self.data = inpt
        self.target = target
        
    def __getitem__(self, index):
        x = self.data[index] 
        x = x[np.newaxis, ...] # To specify the number of channels c (here c=1)
        y = self.target[index]
        
        return {'input': x, 'target': y}
    
    def __len__(self):
        return len(self.data)

dataset = MyDataset()

In [8]:
split = [0.9, 0.1]
split_train = '0.9'
batch_size = 128 
indices = list(range(len(dataset)))
s = int(np.floor(split[1] * len(dataset)))

#shuffling
np.random.seed(111)
np.random.shuffle(indices)
train_indices, val_indices = indices[s:], indices[:s]

train_sampler, val_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=val_sampler)

In [9]:
len(train_dataloader)

1560

In [10]:
np.shape(train_dataloader.dataset.data)

torch.Size([221852, 377])

In [11]:
np.shape(train_dataloader.dataset.data)[1]
f = np.shape(train_dataloader.dataset.data)[1]

In [12]:
dataset_size = len(dataset)

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size=3)
        self.conv2 = nn.Conv1d(n_feature , n_feature, kernel_size=3)
        self.fc1 = nn.Linear(128*42, 128)
        self.fc2 = nn.Linear(128,1)
        
    def forward(self, x): 
        x = self.conv1(x)
        x = F.relu(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()        # Number of input features is 12.
        self.layer_1 = nn.Linear(f, 100) 
        self.layer_2 = nn.Linear(100, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1)
        self.batchnorm2 = nn.BatchNorm1d(32)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        x = F.sigmoid(x)
        
        return x
    
    #layer, relu, bn, dp
    
class LinearModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
                           
    def get_weights(self):
        return self.weight
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = F.sigmoid(self.fc2(out)) #sigmoid as we use BCELoss
        return out

In [13]:
class Net(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            self.layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = F.sigmoid(self.layers[-1](x))
        return out

In [ ]:
model = binaryClassification()
model.to(device)
print(model)
LEARNING_RATE = 0.001
criterion = nn.BCELoss()
optimizer = optim.Adam(model_loop.parameters(), lr=float(LEARNING_RATE))
optimizer_name = 'Adam'
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
dl_arch = 'mlp'
layers= '2hl'
metric = 'bce'

# summary(model_loop, (1, f))

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
dataloaders = {}
dataloaders['train'], dataloaders['val'] = train_dataloader, val_dataloader


In [ ]:
len(dataloaders['train'])

In [ ]:
len(dataloaders['val'])

In [ ]:
len(dataloaders['train'].dataset)

In [ ]:
# for batch_idx, sample in enumerate(dataloaders['train']):
#     print(len(dataloaders['train'].dataset))
#     print(len(dataloaders['train']))

In [ ]:
nb_epoch = 200

In [ ]:
model_name = 'model_'+str(dataset_size)+'spectrum_'+dl_arch+'_'+layers+'_bs'+str(batch_size)+'_lr'+\
    str(LEARNING_RATE)+'_'+str(nb_epoch)+'ep_opt'+str(optimizer_name)+'_split'+split_train+'_'+metric+'_'+params
model_path = '/home/malavika/Documents/Research/assessment/models/model1/'
model_dir = model_path + model_name 
if not os.path.exists(model_dir):
        os.makedirs(model_dir)

In [ ]:
model_name
#model_221852spectrum_mlp_2hl_bs128_lr0.001_200ep_optAdam_split0.9_bce_6

In [ ]:
metrics_path = os.path.join(model_dir, 'metrics.json')
    
metrics = {
    'model': model_dir,
    'optimizer': optimizer.__class__.__name__,
    'criterion': criterion.__class__.__name__,
#     'scheduler': scheduler.__class__.__name__,
    'dataset_size': int(len(dataset)),
    'train_size': int(split[0]*len(dataset)),
    'test_size': int(split[1]*len(dataset)),
    'n_epoch': nb_epoch,
    'batch_size': batch_size,
#     'learning_rate': [],
    'train_loss': [],
    'val_loss': []
}

In [ ]:
#train

def train(n_epochs, model):
    
    best_loss = 0.0
    for epoch in range(n_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            
            #both    
            running_loss = 0.0
            
            for batch_idx, sample in enumerate(dataloaders[phase]):
                inputs = sample['input'].to(device)
                target = sample['target'].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    output = model(inputs)
                    loss = criterion(torch.squeeze(output), torch.squeeze(target))
                    acc = binary_acc(torch.squeeze(output), torch.squeeze(target))
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   
                
                    running_loss += 1 * loss.item() * inputs.size(0) #loss for the phase/whole dataset

                if batch_idx % 100 == 0: 
                    print('{} epoch: {} [{}/{} ({:0.0f}%)]\tLoss: {:.6f}\tAcc: {:.2f}'.format(\
                    phase,epoch,(batch_idx+1)*len(sample['input']),len(dataloaders[phase].dataset),\
                    100.* ((batch_idx+1)*len(sample['input']))/len(dataloaders[phase].dataset),loss.item(), acc))
    
            if phase == 'train':
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[0]))
            else:
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[1]))

            if phase == 'val': 
                if epoch ==  (n_epochs-1) or running_loss < best_loss:
                    print('saving')
                    best_loss = running_loss
                    model_path = os.path.join(model_dir, 'model.pth')
                    torch.save(model.state_dict(), model_path)
                    
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=4)
                    
#         print('--------------------------------------------------------------------')
        

In [ ]:
train(nb_epoch, model)